In [2]:
# Cargando librerias
import gc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

from sklearn.preprocessing import LabelEncoder

from itertools import product
from xgboost import XGBRegressor
from xgboost import plot_importance

## Exploración de datos 2020 - 2021

In [3]:
# Cargando Datos
route = "content/"
LeagueTable = pd.read_excel(route + "soccer2020-2021_final.xlsx", sheet_name="LeagueTable")
PlayerStats = pd.read_excel(route + "soccer2020-2021_final.xlsx", sheet_name="PlayerStats")
Fixtures = pd.read_excel(route + "soccer2020-2021_final.xlsx", sheet_name="Fixtures")
TeamStats_Sort_by_Team = pd.read_excel(route + "soccer2020-2021_final.xlsx", sheet_name="TeamStats_Sort_by_Team")
Lineup_Sort_by_Player = pd.read_excel(route + "soccer2020-2021_final.xlsx", sheet_name="Lineup_Sort_by_Player")

soccer2022 = pd.read_excel(route + "soccer2022-2023_final.xlsx")
soccer2024 = pd.read_excel(route + "soccer2023-2024_final.xlsx")

datos = {"LeagueTable": LeagueTable,
         "PlayerStats": PlayerStats,
         "Fixtures": Fixtures}

#### Info Tables

In [4]:
for nombre, dato in datos.items():
    print("Informacion sobre: " + nombre)
    dato.info()
    print("_"*50)
    print(" ")
    dato.isnull().sum()
    print("_"*50)
    print(" ")

Informacion sobre: LeagueTable
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Team               20 non-null     object
 1   Rank               20 non-null     int64 
 2   Points             20 non-null     int64 
 3   MP                 20 non-null     int64 
 4   Wins               20 non-null     int64 
 5   Ties               20 non-null     int64 
 6   Losses             20 non-null     int64 
 7   GF                 20 non-null     int64 
 8   GA                 20 non-null     int64 
 9   GD                 20 non-null     int64 
 10  Clean Sheets       20 non-null     int64 
 11  Home MP            20 non-null     int64 
 12  Home Wins          20 non-null     int64 
 13  Home Ties          20 non-null     int64 
 14  Home Losses        20 non-null     int64 
 15  Home GF            20 non-null     int64 
 16  Home GA        

## Exploracion PlayerStats

#### General

In [11]:
PlayerStats.head()

,Id,Name,Weight,Height,Age,Citizenship,Team,Jersey,Position,Appearances,...,Own Goals,Yellow Cards,Red Cards,Offsides,Goal Assists,Shots On Target,Total Shots,Total Goals,Goals Conceded,Update Time
3,286,Harry Kane,190.0,74.0,28.0,England,Tottenham Hotspur,10.0,Forward,37.0,...,0.0,5.0,0.0,12.0,9.0,49.0,134.0,17.0,39.0,2022-05-26T22:44:07Z
5,183,Jamie Vardy,161.0,70.0,35.0,England,Leicester City,9.0,Forward,25.0,...,1.0,2.0,0.0,2.0,2.0,26.0,55.0,15.0,26.0,2022-05-26T22:44:06Z
9,350,Raheem Sterling,150.0,67.0,27.0,England,Manchester City,7.0,Forward,30.0,...,0.0,1.0,0.0,6.0,5.0,26.0,54.0,13.0,17.0,2022-05-26T22:44:08Z
10,175,James Maddison,161.0,69.0,25.0,England,Leicester City,10.0,Midfielder,35.0,...,0.0,3.0,0.0,3.0,8.0,29.0,72.0,12.0,41.0,2022-05-26T22:44:06Z
11,320,Ivan Toney,152.0,70.0,26.0,England,Brentford,17.0,Forward,33.0,...,0.0,8.0,0.0,6.0,5.0,26.0,81.0,12.0,45.0,2022-05-26T22:44:12Z


#### Solo England Players

In [36]:
PlayerStats = PlayerStats[PlayerStats["Citizenship"]=="England"]
#len(PlayerStats)
PlayerStats

,Id,Name,Weight,Height,Age,Citizenship,Team,Jersey,Position,Appearances,...,Own Goals,Yellow Cards,Red Cards,Offsides,Goal Assists,Shots On Target,Total Shots,Total Goals,Goals Conceded,Update Time
3,286,Harry Kane,190.0,74.0,28.0,England,Tottenham Hotspur,10.0,Forward,37.0,...,0.0,5.0,0.0,12.0,9.0,49.0,134.0,17.0,39.0,2022-05-26T22:44:07Z
5,183,Jamie Vardy,161.0,70.0,35.0,England,Leicester City,9.0,Forward,25.0,...,1.0,2.0,0.0,2.0,2.0,26.0,55.0,15.0,26.0,2022-05-26T22:44:06Z
9,350,Raheem Sterling,150.0,67.0,27.0,England,Manchester City,7.0,Forward,30.0,...,0.0,1.0,0.0,6.0,5.0,26.0,54.0,13.0,17.0,2022-05-26T22:44:08Z
10,175,James Maddison,161.0,69.0,25.0,England,Leicester City,10.0,Midfielder,35.0,...,0.0,3.0,0.0,3.0,8.0,29.0,72.0,12.0,41.0,2022-05-26T22:44:06Z
11,320,Ivan Toney,152.0,70.0,26.0,England,Brentford,17.0,Forward,33.0,...,0.0,8.0,0.0,6.0,5.0,26.0,81.0,12.0,45.0,2022-05-26T22:44:12Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,565,Antef Tsoungui,NaN,NaN,19.0,England,Brighton & Hove Albion,53.0,Defender,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-26T22:44:09Z
648,565,Mark Helm,NaN,NaN,20.0,England,Burnley,55.0,Midfielder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-26T22:44:10Z
652,625,Dexter Lembikisa,NaN,NaN,18.0,England,Wolverhampton Wanderers,81.0,Defender,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-26T22:44:10Z
656,730,Sammy Braybrooke,NaN,NaN,18.0,England,Leicester City,74.0,Midfielder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-26T22:44:06Z


#### Teams England

In [ ]:

PlayerStats_Team = PlayerStats["Team"].drop_duplicates()
PlayerStats_Team

3            Tottenham Hotspur
5               Leicester City
9              Manchester City
11                   Brentford
12             West Ham United
13                     Arsenal
15                 Aston Villa
18                     Chelsea
23                 Southampton
34                Leeds United
35              Crystal Palace
36            Newcastle United
50      Brighton & Hove Albion
58                     Everton
61           Manchester United
80     Wolverhampton Wanderers
115                    Burnley
147                  Liverpool
190               Norwich City
273                    Watford
Name: Team, dtype: object

#### Role

In [38]:
PlayerStats_Role = PlayerStats["Position"].unique()
PlayerStats_Role

array(['Forward', 'Midfielder', 'Defender', 'Goalkeeper'], dtype=object)

#### Player

In [ ]:
PlayerStats[["Name","Weight","Height","Age","Jersey"]]

,Id,Name,Weight,Height
3,286,Harry Kane,190.0,74.0
5,183,Jamie Vardy,161.0,70.0
9,350,Raheem Sterling,150.0,67.0
10,175,James Maddison,161.0,69.0
11,320,Ivan Toney,152.0,70.0
...,...,...,...,...
647,565,Antef Tsoungui,NaN,NaN
648,565,Mark Helm,NaN,NaN
652,625,Dexter Lembikisa,NaN,NaN
656,730,Sammy Braybrooke,NaN,NaN


## Exploración de TeamStats_Sort_by_Team

#### General

In [16]:
TeamStats_Sort_by_Team.head()

,Team,Fixture,Date,Time,Home Team,Home Goal,Away Goal,Away Team,Possession Pct,Fouls Committed,Yellow Cards,Red Cards,Offsides,Corners Won,Saves,Total Shots,Shots on Target,home/away,Update Time
0,Brentford,Arsenal at Brentford,2021-08-13 15:00:00,2021-08-13 15:00:00,Brentford,2,0,Arsenal,35.3,12,0,0,1,2,4,8,3,home,2022-05-26T04:14:36Z
1,Arsenal,Arsenal at Brentford,2021-08-13 15:00:00,2021-08-13 15:00:00,Brentford,2,0,Arsenal,64.7,8,0,0,1,5,1,22,4,away,2022-05-26T04:14:36Z
2,Manchester United,Leeds United at Manchester United,2021-08-14 07:30:00,2021-08-14 07:30:00,Manchester United,5,1,Leeds United,48.6,11,1,0,2,5,2,16,8,home,2022-05-26T04:14:36Z
3,Leeds United,Leeds United at Manchester United,2021-08-14 07:30:00,2021-08-14 07:30:00,Manchester United,5,1,Leeds United,51.4,9,2,0,3,4,3,10,3,away,2022-05-26T04:14:36Z
4,Watford,Aston Villa at Watford,2021-08-14 10:00:00,2021-08-14 10:00:00,Watford,3,2,Aston Villa,37.9,18,3,0,0,2,0,13,7,home,2022-05-26T04:14:36Z


#### Solo Equipos de England

In [23]:
TeamStats_Sort_by_Team = TeamStats_Sort_by_Team[TeamStats_Sort_by_Team["Team"].isin(PlayerStats_Team)]
TeamStats_Sort_by_Team

,Team,Fixture,Date,Time,Home Team,Home Goal,Away Goal,Away Team,Possession Pct,Fouls Committed,Yellow Cards,Red Cards,Offsides,Corners Won,Saves,Total Shots,Shots on Target,home/away,Update Time
0,Brentford,Arsenal at Brentford,2021-08-13 15:00:00,2021-08-13 15:00:00,Brentford,2,0,Arsenal,35.3,12,0,0,1,2,4,8,3,home,2022-05-26T04:14:36Z
1,Arsenal,Arsenal at Brentford,2021-08-13 15:00:00,2021-08-13 15:00:00,Brentford,2,0,Arsenal,64.7,8,0,0,1,5,1,22,4,away,2022-05-26T04:14:36Z
2,Manchester United,Leeds United at Manchester United,2021-08-14 07:30:00,2021-08-14 07:30:00,Manchester United,5,1,Leeds United,48.6,11,1,0,2,5,2,16,8,home,2022-05-26T04:14:36Z
3,Leeds United,Leeds United at Manchester United,2021-08-14 07:30:00,2021-08-14 07:30:00,Manchester United,5,1,Leeds United,51.4,9,2,0,3,4,3,10,3,away,2022-05-26T04:14:36Z
4,Watford,Aston Villa at Watford,2021-08-14 10:00:00,2021-08-14 10:00:00,Watford,3,2,Aston Villa,37.9,18,3,0,0,2,0,13,7,home,2022-05-26T04:14:36Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,Watford,Watford at Chelsea,2022-05-22 11:00:00,2022-05-22 11:00:00,Chelsea,2,1,Watford,28.3,9,0,0,2,3,6,8,3,away,2022-05-26T04:14:36Z
756,Brighton & Hove Albion,West Ham United at Brighton & Hove Albion,2022-05-22 11:00:00,2022-05-22 11:00:00,Brighton & Hove Albion,3,1,West Ham United,49.9,9,1,0,0,5,1,18,7,home,2022-05-26T04:14:36Z
757,West Ham United,West Ham United at Brighton & Hove Albion,2022-05-22 11:00:00,2022-05-22 11:00:00,Brighton & Hove Albion,3,1,West Ham United,50.1,9,1,0,0,6,4,7,2,away,2022-05-26T04:14:36Z
758,Liverpool,Wolverhampton Wanderers at Liverpool,2022-05-22 11:00:00,2022-05-22 11:00:00,Liverpool,3,1,Wolverhampton Wanderers,62.9,6,1,0,3,5,4,29,8,home,2022-05-26T04:14:36Z
